# HR Diagram for 3 $M_{\odot}$ star

(stole a lot of this code from Earl)

In [1]:
import os

import numpy as np
import scipy as sp
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from ipywidgets import interact, IntSlider

import seaborn as sns

In [501]:
%config InlineBackend.figure_format = 'retina'

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 24

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': 0.6*fs,
          'axes.labelsize': 0.8*fs,
          'xtick.labelsize': 0.6 * fs,
          'ytick.labelsize': 0.6 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)

In [3]:
from importlib import reload
import sys
sys.path.append("../")

import mesagrid

In [149]:
usecols_history = ['model_number', 'star_age', 'log_Teff', 'log_L',
                   'center_h1', 'center_he4']
cpus = 8

track = mesagrid.Track("../template",
                       usecols_history=usecols_history,
                       cpus=cpus)

In [ ]:
spectras

In [226]:
np.diff(np.log10(add_spectral_background(ax)[-1]))

array([-0.30103   , -0.12493874, -0.09691001, -0.06214791, -0.01703334])

In [229]:
10**(np.log10(add_spectral_background(ax)[-1][1:]) + np.diff(np.log10(add_spectral_background(ax)[-1])))

array([3.69897   , 3.75012253, 3.68124124, 3.65385544, 3.68193667])

In [249]:
types = ['X', 'O', 'B', 'A', 'F', 'G', 'K', 'M']
spectrals = np.array([1e99, 30000, 10000, 7500, 6000, 5200, 3700, 2400])
rgbs = [(1, 1, 1), # X, temp class just for setting upper bound 
        (175/255, 201/255, 1),       # O
        (199/255, 216/255, 1),       # B
        (1,       244/255, 243/255), # A 
        (1,       229/255, 207/255), # F 
        (1,       217/255, 178/255), # G 
        (1,       199/255, 142/255), # K 
        (1,       166/255, 81/255)]  # M

def add_spectral_background(ax):
    last = 1e6
    for spectral, rgb in zip(spectrals, rgbs):
        ax.fill_between([last, spectral], [1,1], [1e6,1e6], color=rgb, zorder=-99)
        last = spectral
    top = ax.twiny()
    top.set_xlabel(r'Spectral class', labelpad=10)
    
    # logic to determine which labels to include 
    xmax, xmin = ax.get_xlim()
    types_ = [] 
    spectrals_ = [] 
    for i in range(len(spectrals)):
        if spectrals[i] >= xmax: 
            if i+1 <= len(spectrals) and spectrals[i+1] < xmax:
                spectrals_ += [xmax]
                types_ += [types[i]]
        elif spectrals[i] <= xmin: 
            spectrals_ += [xmin]
            types_ += [types[i]]
            break
        else:
            spectrals_ += [spectrals[i]]
            types_ += [types[i]]
    
    top.set_xticks([], minor=True)
    top.set_xticks([ 10**((np.log10(spectrals_[k])+np.log10(spectrals_[k+1]))/2) 
                     for k in range(len(spectrals_)-1) ])
    print(top.get_xticks())
    
    top.set_xticklabels(types_[1:]) 
    top.set_xlim(ax.get_xlim())
    
    return ax, spectrals_

In [ ]:
np.

In [248]:
np.logspace(np.log10(20000), np.log10(10000), 3)

array([20000.        , 14142.13562373, 10000.        ])

In [244]:
10**((np.log10(30) + np.log10(10)) / 2)

17.320508075688775

In [ ]:
np.log10(30)

1.4771212547196624

In [ ]:
np.log10(10)

1.0

In [ ]:
np.logspace(np.log10(10), np.log10(30), 3)

array([10.        , 17.32050808, 30.        ])

In [250]:
add_spectral_background(ax)[-1]

[14142.13562373  8660.25403784  6708.2039325   5585.69601751
  5099.01951359]


[20000.0, 10000.0, 7500.0, 6000.0, 5200.0, 5000.0]

In [209]:
def add_radius_lines(ax, Rs=[10, 100], Tlower=1, Tupper=1e6, Lpos=6e3, 
                     sigma=5.67e-5, Lsol=3.839e33, Rsol=6.955e10):
    L_ = lambda R, Teff: 4*np.pi*(R*Rsol)**2*sigma*Teff**4 / Lsol
    T_ = lambda R, L: np.power(L*Lsol/(4*np.pi*(R*Rsol)**2*sigma), 1/4)
    for R in Rs:
        ax.text(T_(R, Lpos) + 0.03, Lpos*0.75, 
                 str(R)+r' R$_\odot$', c='gray', size=16, weight='bold', rotation=-68)
        ax.plot([Tlower,        Tupper], 
                         [L_(R, Tlower), L_(R, Tupper)], 
                 ls='--', c='k', lw=2, zorder=0)
        
    return ax

In [12]:
norm = mpl.colors.Normalize(vmin=-0.5, vmax=5.5)
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Greys_r)
vmin = int(norm.vmin)
vmax = int(norm.vmax)

In [477]:
def simple_hr(track, label='', ylabel=r'Luminosity $\mathbf{L/L_{\odot}}$',
              cbar_var="center_he4", cbar_label=r"$X_{\rm He, center}$",
              fig=None, ax=None, show=True, **kwargs):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(8,6))
    
    DF_ = track.history
    if 's' not in kwargs:
        kwargs['s'] = 5
    ax.scatter(DF_['log_Teff'], DF_['log_L'], 
               c=DF_[cbar_var], **kwargs)
    
    if label != '':
        ax.annotate(label, xy=(0.97, 0.97), xycoords="axes fraction",
                    weight='bold', size=0.8 * fs,
                    ha='right', va='top')
    # ax.set_ylim([5e1, 2e2])
    # ax.set_xlim([2e4, 5e3])
    ax.invert_xaxis()
    ax.set_xlabel(r'Effective temperature $\mathbf{T_{eff}/K}$')
    ax.set_ylabel(ylabel)
    
    inset_ax = ax.inset_axes([0.48, 0.05, 0.5, 0.05])
    fig.colorbar(ax.collections[0], label=cbar_label, cax=inset_ax, orientation="horizontal", location="top")
    
    # plt.tight_layout()
    
    if show:
        plt.show()
        
    return fig, ax

In [261]:
from matplotlib.gridspec import GridSpec

In [516]:
def compare_composition(track, label='', ylabel=r'Luminosity $\mathbf{L/L_{\odot}}$',
            cbar_var="center_he4", cbar_label=r"$X_{\rm He, center}$",
            show=True, model_nums=[1, 50, 100], **kwargs):
    n_comps = len(model_nums)
    
    fig = plt.figure(figsize=(18 / 3 * n_comps, 5.5))
    gs = GridSpec(n_comps, n_comps, figure=fig, hspace=0.0)
    ax_hr = fig.add_subplot(gs[:, 0])
    ax_comp = [fig.add_subplot(gs[i, 1]) for i in range(n_comps)]

    fig, ax = simple_hr(track, label=label, ylabel=ylabel, cbar_var=cbar_var, cbar_label=cbar_label,
                        show=False, fig=fig, ax=ax_hr, **kwargs)
    
    lw = 3
    H_col = plt.get_cmap("cividis")(0.3)
    He_col = plt.get_cmap("cividis")(0.9)
    for i, args in enumerate(zip(model_nums, ax_comp)):
        model_num, ax = args
        
        lTeff, lL = track.history.loc[model_num - 1]["log_Teff"], track.history.loc[model_num - 1]["log_L"]
        ax_hr.scatter(lTeff, lL,
                      s=50, facecolors="none", color="red")
        ax_hr.annotate(chr(ord('a') + i), xytext=(lTeff+0.01, lL - 0.005), xy=(lTeff, lL),
                       arrowprops=dict(arrowstyle="-|>", color="black"),
                       ha="center", va="center", fontsize=0.6*fs)
        
        ax.annotate(chr(ord('a') + i), xy=(0.97, 0.95),
                    xycoords="axes fraction", ha="right", va="top", fontsize=0.6*fs,
                    bbox=dict(facecolor="lightgrey", boxstyle="circle"))
        ax.annotate(f'Star Age: {track.history.loc[model_num - 1]["star_age"] / 1e6:1.1f}Myr', xy=(0.5, 0.95),
                    xycoords="axes fraction", ha="center", va="top", fontsize=0.6*fs)               
        
        ax.plot(track.profiles[model_num - 1]["mass"], track.profiles[model_num - 1]["x_mass_fraction_H"],
                lw=lw, color=H_col, label="Hydrogen")
        ax.plot(track.profiles[model_num - 1]["mass"], track.profiles[model_num - 1]["y_mass_fraction_He"],
                lw=lw, color=He_col, label="Helium")
        ax.set_xlim(0, 3)
        ax.set_ylim(0, 1)
        
        for el in ["center_h1", "center_he4"]:
            ax.axhline(track.history.iloc[0][el], color="grey", linestyle="--", lw=0.5, zorder=-1)

        if ax != ax_comp[-1]:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel(r"Mass $\mathbf{M/M_{\odot}}$")
            ax.legend(loc="lower right", ncol=2, fontsize=0.4*fs)
    
    if show:
        plt.show()
        
    return fig, ax

In [526]:
def interact_comp(a, b):
    compare_composition(track, model_nums=[a, b], cbar_var="center_h1", cbar_label=r"$X_{\rm H, center}$", show=False)

In [435]:
model_min, model_max = track.history["model_number"].min(), track.history["model_number"].max()

In [527]:
interact(interact_comp,
         a=IntSlider(value=50, min=model_min, max=model_max, continuous_update=False),
         b=IntSlider(value=100, min=model_min, max=model_max, continuous_update=False));

interactive(children=(IntSlider(value=50, continuous_update=False, description='a', max=116, min=1), IntSlider…

In [252]:
def plot_hr(track, label='', ylabel=r'Luminosity $\mathbf{L/L_{\odot}}$',
            cbar_var="center_he4", cbar_label=r"$X_{\rm He, center}$",
            fig=None, ax=None, show=True, **kwargs):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(8,6))
    
    DF_ = track.history
    if 's' not in kwargs:
        kwargs['s'] = 5
    ax.scatter(10**DF_['log_Teff'], 10**DF_['log_L'], 
               c=DF_[cbar_var], **kwargs)
    
    if label != '':
        ax.annotate(label, xy=(0.97, 0.97), xycoords="axes fraction",
                    weight='bold', size=0.8 * fs,
                    ha='right', va='top')
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_ylim([5e1, 2e2])
    ax.set_xlim([2e4, 5e3])
    # ax.invert_xaxis()
    ax.set_xlabel(r'Effective temperature $\mathbf{T_{eff}/K}$')
    ax.set_ylabel(ylabel)
    
    # fig.colorbar(cmap, label=r'log $\mathbf{v_{HR}}$', 
    #              boundaries=np.array(range(vmin, vmax+2, 1)) - 0.5, 
    #              ticks=range(vmin, vmax+1, 1), ax=ax)
    
    fig.colorbar(ax.collections[0], label=cbar_label, ax=ax)

    add_radius_lines(ax, [2, 5], Lpos=2e2)
    add_spectral_background(ax)
    
    # ax.set_xticks([], minor=True)
    # ax.set_xticks([15000, 10000, 7500])
    # ax.set_xticklabels(str(int(x)) for x in np.round(ax.get_xticks(), -1))
    
    plt.tight_layout()
    
    if show:
        plt.show()
        
    return fig, ax